# Implementation of an bully algorithm without the Django framework
### This is a test, to know how it's build without the extra layers -> object based Python classes

In [5]:
from random import randint
from threading import Thread

import numpy as np

In [6]:
__name__ = '__main__'

AMOUNT_OF_TRUCKS = 5
INITIAL_SPEED = 30.0 # km/h
INITIAL_BROKE = False

TRUCK_IDS = []
TRUCKS = []
CONVOY_LIST = False

In [7]:
class TruckThreading(Thread):
    class __Truck__(object):
        def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broken: bool, convoyReferences: object):
            self.truckId = truckId
            self.convoyPosition = convoyPosition
            self.convoyLeaderId = convoyLeaderId
            self.speed = speed
            self.broken = broken
            self.convoyReferences = convoyReferences # Aktuell ein Object, ist spaeter lediglich eine Adresse im Hostingsystem hinter der ein RMI aufgwerufen wird, welcher die Adressen der jeweiligen Trucks zurueck gibt

    def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broke: bool, convoyReferences: object):
        Thread.__init__(self)
        self.__alive__ = True
        self.__truck__ = self.__Truck__(truckId, convoyPosition, convoyLeaderId, speed, broke, convoyReferences)
    
    def run(self):
        while self.__alive__:
            pass
    
    def stop(self):
        self.__alive__ = False
    
    def broken(self):
        return self.__truck__.broken

    def toggleBroken(self):
        self.__truck__.broken = not self.__truck__.broken
        
    def getTruckId(self):
        return self.__truck__.truckId
    
    def setConvoyPosition(self, convoyPosition):
        self.__truck__.convoyPosition = convoyPosition
        
    def getConvoyPosition(self):
        return self.__truck__.convoyPosition
    
    def setConvoyLeaderId(self, convoyLeaderId):
        self.__truck__.convoyLeaderId = convoyLeaderId
        
    def getConvoyLeaderId(self):
        return self.__truck__.convoyLeaderId
    
    def setSpeed(self, speed):
        self.__truck__.speed = speed
        
    def getSpeed(self):
        return self.__truck__.speed
    
    def startPoll(self): # Wird einfacher mit Microservice, programmiere ich so jetzt nicht aus... muss auch global laufen
        print(f'START: {self.getConvoyPosition()}')
        
        trucksInFront = []
        trucks = TRUCK_IDS[:self.getConvoyPosition()]
        
        for currentId, currentPosition in trucks:
            current = TRUCKS[0][currentPosition]
            if not current.broken():
                positionsAlive = current.startPoll()
                trucksInFront.append(positionsAlive)
        
        position = len(trucksInFront)
        self.setConvoyPosition(position)
        
        if position == 0:
            print("ICH BIN DER VERFICKTE LEADER")
        
        print(f'STOP: {self.getConvoyPosition()}')
        
        return position


In [8]:
class ConvoyListThreading(Thread):
    class __ConvoyReferenceList__(object):
        def __init__(self):
            self.convoyList = {}
    
    def __init__(self):
        Thread.__init__(self)
        self.__alive__ = True
        self.__convoyReferenceList__ = self.__ConvoyReferenceList__()
    
    def run(self):
        while self.__alive__:
            pass
    
    def stop(self):
        self.__alive__ = False
    
    def getAllTruckAdresses(self):
        """@return list of adresses of the trucks in this convoy"""
        return self.__convoyReferenceList__.convoyList.keys()
    
    def getAllTruckIds(self):
        """@return list of ids of the trucks in this convoy"""
        return self.__convoyReferenceList__.convoyList.values()
    
    def getTruckAdress(self, truckId: int):
        """@return resolved networkadress for the requested truckId"""
        return self.__convoyReferenceList__.convoyList.get(truckId)
    
    def addTuck(self, truckId: int, adress: str):
        """@return success"""
        if not truckId in self.__convoyReferenceList__.convoyList.values():
            self.__convoyReferenceList__.convoyList[truckId] = adress
            return True
        return False
        
    def removeTruck(self, truckId: int):
        """@return success"""
        if truckId in self.__convoyReferenceList__.convoyList.values():
            del self.__convoyReferenceList__.convoyList[truckId]
            return True
        return False


In [9]:
def initTruckIdList(counter: int):
    truckIdList = []
    for count in range(counter):
        newConvoyPosition = count
        newTruckId = randomTruckId(truckIdList)
        listEntry = (newTruckId, newConvoyPosition)
        print(listEntry)
        truckIdList.append(listEntry)
    return truckIdList

def initTrucks(truckIdList: list, initialSpeed: float, initialBroke: bool, convoyReferenceList: object):
    leaderId = 0
    truckIds = []
    trucks = []
    inc = 0
    
    for newTruckId, newConvoyPosition in truckIdList:
        truckIds.append(newTruckId)
        
        if newConvoyPosition == 0:
            leaderId = newTruckId
        newTruck = TruckThreading(truckId = newTruckId, convoyPosition = newConvoyPosition, convoyLeaderId = leaderId, speed = initialSpeed, broke = initialBroke, convoyReferences = convoyReferenceList)
        trucks.append(newTruck)
        inc += 1
    return (trucks, truckIds)
    
def initConvoyList(truckIdList: list):
    convoyList = ConvoyListThreading()
    for truckId, convoyPosition in truckIdList:
        adress = convoyPosition
        success = convoyList.addTuck(truckId, adress)
        if not success:
            raise ValueError(f'Can\'t reach truck with id \'{truckId}\' and adress \'{adress}\'')
    return convoyList

def randomTruckId(truckIds: list):
    success = False
    while not success:
        newId = randint(1, 1000)
        if not newId in truckIds:
            success = not success
    return newId


In [12]:
if __name__ == '__main__':
    TRUCK_IDS = initTruckIdList(AMOUNT_OF_TRUCKS)
    CONVOY_LIST = initConvoyList(TRUCK_IDS)
    TRUCKS = initTrucks(TRUCK_IDS, INITIAL_SPEED, INITIAL_BROKE, CONVOY_LIST)
    current = TRUCKS[0][3]
    current.startPoll()

# https://www.youtube.com/watch?v=Tf1DEI2lEe0


(285, 0)
(638, 1)
(283, 2)
(492, 3)
(937, 4)
START: 3
START: 0
ICH BIN DER VERFICKTE LEADER
STOP: 0
START: 1
START: 0
ICH BIN DER VERFICKTE LEADER
STOP: 0
STOP: 1
START: 2
START: 0
ICH BIN DER VERFICKTE LEADER
STOP: 0
START: 1
START: 0
ICH BIN DER VERFICKTE LEADER
STOP: 0
STOP: 1
STOP: 2
STOP: 3
